<a href="https://colab.research.google.com/github/HuyenNguyenHelen/LING-5412/blob/main/Assignment5_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
import torch
import sklearn
from sklearn.metrics import accuracy_score, classification_report
tfds.disable_progress_bar()

In [ ]:
# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')



# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla K80


# Loading the dataset

In [ ]:
dataset, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteV5KD2M/imdb_reviews-train.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteV5KD2M/imdb_reviews-test.tfrecord
Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteV5KD2M/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [ ]:
info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [ ]:
# Shuffling the dataset
buffer_size = 10000
batch_size = 64
train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Building the model

## Building the LSTM model

In [ ]:
# Defining an evaluation metric function
def printing_eval_scores (y_true, y_pred, report=''):
  accuracy = sklearn.metrics.accuracy_score(y_true, y_pred)
  precision = sklearn.metrics.precision_score(y_true, y_pred, average='binary')
  recall = sklearn.metrics.recall_score(y_true, y_pred, average='binary')
  f1 = sklearn.metrics.f1_score(y_true, y_pred , average='binary')
  print('accuracy score: {:.3f}'.format(accuracy))
  print('precision score: {:.3f}'.format(precision))
  print('recall score: {:.3f}'.format(recall))
  print('F1 score: {:.3f}'.format(f1))
  if report is True:
    print(classification_report(y_true, y_pred))
  else:
    pass
  return accuracy, precision, recall, f1

### With different embedding sizes

In [ ]:
## Representing the  text
vocab_size = 10000
encoder = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
encoder.adapt(train_dataset.map(lambda x,y: x))

# Store vocabulary
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U17')

In [ ]:
# Creating the model
embedding_sizes = [32,64,128]
for size in embedding_sizes:
  print ("\n========= embedding vectors'size= %s ============" %size)
  model = tf.keras.Sequential([encoder,
                              tf.keras.layers.Embedding(
                                  input_dim = len(encoder.get_vocabulary()),
                                  output_dim = size,
                                  mask_zero = True),
                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                              tf.keras.layers.Dense(64, activation = 'relu'),
                              tf.keras.layers.Dense(1, activation = 'sigmoid')]) 
  print(model.summary())
  # Compile the model for training
  model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = False),
                optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
                metrics = ['accuracy'])
  # Training the model
  history = model.fit (train_dataset,
                      epochs = 10, 
                      validation_data = test_dataset,
                      validation_steps = 30)
  # testing the model
  ### pred_label = tf.argmax(model.predict(test),1)
  pred_label = (model.predict(test_dataset) > 0.5).astype("int32")
  true_label = np.concatenate([y for x, y in test], axis=0)

  test_loss, test_acc = model.evaluate (test_dataset)
  # print('Test loss: ', test_loss)
  # print('Test acurracy: ', test_acc)
  print('\nTesting performance:\n Loss: {:.3f} - Accuracy: {:.3f}'. format(test_loss, test_acc))
  printing_eval_scores (true_label, pred_label, report=True)



Epoch 1/10
391/391 [==============================] - 109s 251ms/step - loss: 0.6309 - accuracy: 0.6450 - val_loss: 0.4827 - val_accuracy: 0.7969
Epoch 2/10
391/391 [==============================] - 94s 238ms/step - loss: 0.3926 - accuracy: 0.8348 - val_loss: 0.4405 - val_accuracy: 0.8016
Epoch 3/10
391/391 [==============================] - 93s 235ms/step - loss: 0.3033 - accuracy: 0.8807 - val_loss: 0.3374 - val_accuracy: 0.8708
Epoch 4/10
391/391 [==============================] - 95s 239ms/step - loss: 0.2338 - accuracy: 0.9110 - val_loss: 0.3155 - val_accuracy: 0.8708
Epoch 5/10
391/391 [==============================] - 92s 233ms/step - loss: 0.1947 - accuracy: 0.9294 - val_loss: 0.3460 - val_accuracy: 0.8776
Epoch 6/10
391/391 [==============================] - 93s 234ms/step - loss: 0.1676 - accuracy: 0.9414 - val_loss: 0.3469 - val_accuracy: 0.8734
Epoch 7/10
391/391 [==============================] - 92s 232ms/step - loss: 0.1411 - accuracy: 0.9526 - val_loss: 0.3368 - val_a

### With different vocabulary size

In [ ]:
vocab_sizes = [5000, 7000, 10000]
for size in vocab_sizes:
  print ("\n========= vocabulary size = %s ============" %size)
  encoder = tf.keras.layers.TextVectorization(max_tokens=size)
  encoder.adapt(train_dataset.map(lambda x,y: x))
  # Store vocabulary
  vocab = np.array(encoder.get_vocabulary())

  model = tf.keras.Sequential([encoder,
                              tf.keras.layers.Embedding(
                                  input_dim = len(encoder.get_vocabulary()),
                                  output_dim = 32,
                                  mask_zero = True),
                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
                              tf.keras.layers.Dense(32, activation = 'relu'),
                              tf.keras.layers.Dense(1, activation = 'sigmoid')]) 
  print(model.summary())
  # Compile the model for training
  model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = False),
                optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
                metrics = ['accuracy'])
  # Training the model
  history = model.fit (train_dataset,
                      epochs = 10, 
                      validation_data = test_dataset,
                      validation_steps = 30)
  # testing the model
  ### pred_label = tf.argmax(model.predict(test),1)
  pred_label = (model.predict(test_dataset) > 0.5).astype("int32")
  true_label = np.concatenate([y for x, y in test_dataset], axis=0)

  test_loss, test_acc = model.evaluate (test_dataset)
  # print('Test loss: ', test_loss)
  print('Test acurracy: ', test_acc)
  print('\nTesting performance:\n Loss: {:.3f} - Accuracy: {:.3f}'. format(test_loss, test_acc))
  printing_eval_scores (true_label, pred_label, report=True)


========= vocabulary size = 5000 ============
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 32)          160000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              49664     
 l)                                                              
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total par

NameError: ignored

### With different optimizers

In [ ]:
# configure the model uisng optimizer and loss function
optimizers = ['adagrad', 'rmsprop', 'adam']

print(model.summary())
for opt in optimizers:
  print( '\n========== optimizer = %s' %opt)
  # Compile the model for training
  model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = False),
                optimizer = opt,
                metrics = ['accuracy'])
  # Training the model
  history = model.fit (train_dataset,
                      epochs = 10, 
                      validation_data = test_dataset,
                      validation_steps = 30)
  # testing the model
  ### pred_label = tf.argmax(model.predict(test),1)
  pred_label = (model.predict(test_dataset) > 0.5).astype("int32")
  true_label = np.concatenate([y for x, y in test_dataset], axis=0)

  test_loss, test_acc = model.evaluate (test_dataset)
  # print('Test loss: ', test_loss)
  print('Test acurracy: ', test_acc)
  print('\nTesting performance:\n Loss: {:.3f} - Accuracy: {:.3f}'. format(test_loss, test_acc))
  printing_eval_scores (true_label, pred_label, report=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 32)          160000    
                                                                 
 bidirectional (Bidirectiona  (None, 128)              49664     
 l)                                                              
                                                                 
 dense (Dense)               (None, 32)                4128      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 213,825
Trainable params: 213,825
Non-trai

### Replacing with LSTM with GRU 

In [ ]:
model = tf.keras.Sequential([encoder,
                              tf.keras.layers.Embedding(
                                  input_dim = len(encoder.get_vocabulary()),
                                  output_dim = 32,
                                  mask_zero = True),
                              tf.keras.layers.GRU(64,
                                                  activation = 'tanh',
                                                  recurrent_activation = 'sigmoid',
                                                  recurrent_dropout = 0.0,
                                                  use_bias = True),
                              tf.keras.layers.Dense(32, activation = 'relu'),
                              tf.keras.layers.Dense(1, activation = 'sigmoid')]) 
print(model.summary())
# Compile the model for training
print( 'Training GRU model...')
model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = False),
              optimizer = 'adagrad',
              metrics = ['accuracy'])
# Training the model
history = model.fit (train_dataset,
                    epochs = 10, 
                    validation_data = test_dataset,
                    validation_steps = 30)
# testing the model
### pred_label = tf.argmax(model.predict(test),1)
pred_label = (model.predict(test_dataset) > 0.5).astype("int32")
true_label = np.concatenate([y for x, y in test_dataset], axis=0)

test_loss, test_acc = model.evaluate (test_dataset)
print('Test acurracy: ', test_acc)
print('\nTesting performance:\n Loss: {:.3f} - Accuracy: {:.3f}'. format(test_loss, test_acc))
printing_eval_scores (true_label, pred_label, report=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 32)          320000    
                                                                 
 gru (GRU)                   (None, 64)                18816     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 340,929
Trainable params: 340,929
Non-trainable params: 0
__________________________________________________

NameError: ignored

# Stacking 2 bidirectional LSTM

In [ ]:
# Creating the model
model = tf.keras.Sequential([encoder,
                             tf.keras.layers.Embedding(
                                 input_dim = len(encoder.get_vocabulary()),
                                 output_dim = 64,
                                 mask_zero = True),
                            tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True),
                                                          backward_layer = tf.keras.layers.LSTM(64, return_sequences = True, go_backwards = True),
                                                          merge_mode = 'concat'),
                            tf.keras.layers.Dense(64, activation = 'relu'),
                            tf.keras.layers.Dense(1, activation = 'sigmoid')])


# Compile the model for training
model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = False),
              optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics = ['accuracy'])

# Training the model
history = model.fit (train_dataset,
                    epochs = 10, 
                    validation_data = test_dataset,
                    validation_steps = 30)
# testing the model
### pred_label = tf.argmax(model.predict(test),1)
pred_label = (model.predict(test_dataset) > 0.5).astype("int32")
true_label = np.concatenate([y for x, y in test_dataset], axis=0)

test_loss, test_acc = model.evaluate (test_dataset)
print('Test acurracy: ', test_acc)
print('\nTesting performance:\n Loss: {:.3f} - Accuracy: {:.3f}'. format(test_loss, test_acc))
printing_eval_scores (true_label, pred_label, report=True)

Epoch 1/10


ValueError: ignored

### With an average of all hidden states to fully connected layer

In [ ]:
# Creating the model
model = tf.keras.Sequential([encoder,
                              tf.keras.layers.Embedding(
                                  input_dim = len(encoder.get_vocabulary()),
                                  output_dim = 64,
                                  mask_zero = True),
                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences = True)),
                             tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1)),
                             tf.keras.layers.GlobalAveragePooling1D(),
                              tf.keras.layers.Dense(1, activation = 'sigmoid')]) 

# Compile the model for training
model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = False),
              optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics = ['accuracy'])

# Training the model
history = model.fit (train_dataset,
                    epochs = 5, 
                    validation_data = test_dataset,
                    validation_steps = 30)
# testing the model
### pred_label = tf.argmax(model.predict(test),1)
pred_label = (model.predict(test_dataset) > 0.5).astype("int32")
true_label = np.concatenate([y for x, y in test_dataset], axis=0)

test_loss, test_acc = model.evaluate (test_dataset)
print('Test acurracy: ', test_acc)
print('\nTesting performance:\n Loss: {:.3f} - Accuracy: {:.3f}'. format(test_loss, test_acc))
printing_eval_scores (true_label, pred_label, report=True)

Epoch 1/5
391/391 [==============================] - 109s 253ms/step - loss: 0.5796 - accuracy: 0.7092 - val_loss: 0.5005 - val_accuracy: 0.7984
Epoch 2/5
391/391 [==============================] - 96s 242ms/step - loss: 0.4250 - accuracy: 0.8394 - val_loss: 0.4192 - val_accuracy: 0.8313
Epoch 3/5
391/391 [==============================] - 94s 238ms/step - loss: 0.3477 - accuracy: 0.8588 - val_loss: 0.3764 - val_accuracy: 0.8490
Epoch 4/5
391/391 [==============================] - 94s 237ms/step - loss: 0.2872 - accuracy: 0.8908 - val_loss: 0.3806 - val_accuracy: 0.8469
Epoch 5/5
391/391 [==============================] - 51s 129ms/step - loss: 0.3565 - accuracy: 0.8623
Test acurracy:  0.8623200058937073

Testing performance:
 Loss: 0.357 - Accuracy: 0.862
accuracy score: 0.862
precision score: 0.872
recall score: 0.849
F1 score: 0.860
              precision    recall  f1-score   support

           0       0.85      0.88      0.86     12500
           1       0.87      0.85      0.86

(0.86232, 0.8721446179129005, 0.84912, 0.8604783137413863)

In [ ]:

print(model.summary())


NameError: ignored

In [ ]:
# Creating the model
model = tf.keras.Sequential([encoder,
                              tf.keras.layers.Embedding(
                                  input_dim = len(encoder.get_vocabulary()),
                                  output_dim = 64,
                                  mask_zero = True),
                              tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences = True)),
                            # tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1)),
                             tf.keras.layers.GlobalAveragePooling1D(),
                              tf.keras.layers.Dense(1, activation = 'sigmoid')]) 

print(model.summary())
# Compile the model for training
model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = False),
              optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics = ['accuracy'])

# Training the model
history = model.fit (train_dataset,
                    epochs = 5, 
                    validation_data = test_dataset,
                    validation_steps = 30)
# testing the model
### pred_label = tf.argmax(model.predict(test),1)
pred_label = (model.predict(test_dataset) > 0.5).astype("int32")
true_label = np.concatenate([y for x, y in test_dataset], axis=0)

test_loss, test_acc = model.evaluate (test_dataset)
print('Test acurracy: ', test_acc)
print('\nTesting performance:\n Loss: {:.3f} - Accuracy: {:.3f}'. format(test_loss, test_acc))
printing_eval_scores (true_label, pred_label, report=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 64)          640000    
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        66048     
 l)                                                              
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                        

(0.85764, 0.8620717583218596, 0.85152, 0.8567633919587878)

## With attention layers

In [ ]:
from keras import backend as K
units = 64
max_length = 50
_input = tf.keras.Input(shape = [max_length], dtype = 'int32')
embeddings = tf.keras.layers.Embedding(
                                  input_dim = len(encoder.get_vocabulary()),
                                  output_dim = 64,
                                  input_length=max_length,
                                  mask_zero = True)(_input)
activations = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences = True)) (embeddings)

attention = tf.keras.layers.Dense(1, activation = 'tanh') (activations)
attention = tf.keras.layers.Flatten()(attention)
attention = tf.keras.layers.Activation('softmax') (attention)
attention = tf.keras.RepeatVector (units*2) (attention)
attention = tf.keras.layers.Permute ([2,1]) (attention)

attention_weight = tf.keras.layers.Multiply()([activations, attention])
attention_weight = tf.keras.layers.Lambda(lambda x: K.sum(x, axis = 1)) (attention_weight)
output_layer = tf.keras.layers.Dense(1, activation = 'sigmoid') (attention_weight)

model = tf.keras.Model(inputs = _input, outputs = output_layer)
# model = tf.keras.Sequential([encoder,
#                               tf.keras.layers.Embedding(
#                                   input_dim = len(encoder.get_vocabulary()),
#                                   output_dim = 64,
#                                   mask_zero = True),
#                               tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences = True)),
#                               #tf.keras.layers.Attention(name='attention_weight'),
#                               #tf.keras.layers.SeqSelfAttention(attention_activation='sigmoid'),
#                               #tf.keras.layers.Flatten(),
#                               Attention(),
#                               tf.keras.layers.Dense(32, activation='relu'),
#                               tf.keras.layers.Dense(1, activation = 'sigmoid')]) 

print(model.summary())
# Compile the model for training
model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = False),
              optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4),
              metrics = ['accuracy'])

# Training the model
history = model.fit (train_dataset,
                    epochs = 5, 
                    validation_data = test_dataset,
                    validation_steps = 30)
# testing the model
### pred_label = tf.argmax(model.predict(test),1)
pred_label = (model.predict(test_dataset) > 0.5).astype("int32")
true_label = np.concatenate([y for x, y in test_dataset], axis=0)

test_loss, test_acc = model.evaluate (test_dataset)
print('Test acurracy: ', test_acc)
print('\nTesting performance:\n Loss: {:.3f} - Accuracy: {:.3f}'. format(test_loss, test_acc))
printing_eval_scores (true_label, pred_label, report=True)

TypeError: ignored